In [ ]:
#!pip install pandas==1.3.5 # for comaparability woth colab

In [ ]:
#%pip install levenshtein

In [1]:
import os 
import pickle 
import pandas as pd
import yaml
import sys
from datetime import date

import importlib
sys.path.append('../3-data-preprocessing')
import preprocessing_utils
importlib.reload(preprocessing_utils)

<module 'preprocessing_utils' from '/usr/net/zinovyee.hub/IRTG/Encode-the-Qode/Quantlet/2-Parse-Quantlets-Folders/../3-data-preprocessing/preprocessing_utils.py'>

In [26]:
# PATHS 
PATH_TO_PARSE = '../../data/QuantLet'
FILE_TYPES = ['m', 'py', 'r', 'R', 'M', 'ipynb']
TEST_PATH = '../data/Q_test'

# CONSTANTS
IPY_CONVERT = False

# VARIABLE PARAMETERS
TODAY = date.today().strftime("%d%m%Y")

In [27]:
if IPY_CONVERT:
    # q_name, folder_name, code_script, type_script, metainfo_file
    language, name = '', ''
    for i, (root, directories, files) in enumerate(os.walk(PATH_TO_PARSE)):
        
        for file in files:
            language = file.split('.')[-1]
            
            if (language == 'ipynb'):
                if not os.path.exists(f"{root}/{file.replace('ipynb', 'py')}"):
                    print(file)
                    os.popen(f"jupyter nbconvert --to python {root}/{file}")
            else:
                continue    

In [28]:
Quantlets = []
repos = []
nw = 0
code_file_counter = 0
for i, (root, directories, files) in enumerate(os.walk(PATH_TO_PARSE)):
        
        if '.git' in root:
             continue
        print(f'Processing {i} folder: {root}...')


        m_file = [file for file in files if file.lower()=="metainfo.txt"] 
        #q = {}
        
        #folder_name = root
        
        if len(m_file)!=0:
            
            with open(f'{root}/{m_file[0]}', 'r') as meta:
                try:
                    metainfofile = yaml.safe_load(meta)#.readlines()
                except:
                    nw += 1
            #q['metainfo_file'] = metainfofile
        

            code_files = [file for file in files if file.split('.')[-1] in FILE_TYPES] 
            
            
            if len(code_files)!=0:
                
                for code_i, c_file in enumerate(code_files):
                    
                    q = {}

                    language = c_file.split('.')[-1].lower()

                    try:
                        sc_file = preprocessing_utils.load_code(f'{root}/{c_file}', language=language)
                    except:
                        sc_file = 'empty'
                     
                    q['folder_name'] = root
                    q['metainfo_file'] = metainfofile
                    q['code_script'] = sc_file
                    q['type_script'] = language
                    q['script_name'] = c_file
                    repos.append(q)
                    code_file_counter += 1 

Processing 0 folder: ../../data/QuantLet...
Processing 1 folder: ../../data/QuantLet/FRM_Quantlet_China...
Processing 19 folder: ../../data/QuantLet/FRM_Quantlet_China/FRM_All...
Processing 20 folder: ../../data/QuantLet/FRM_Quantlet_China/FRM_All/Cluster...
Processing 21 folder: ../../data/QuantLet/FRM_Quantlet_China/FRM_All/Output 20210727...
Processing 22 folder: ../../data/QuantLet/FRM_Quantlet_China/FRM_All/Output 20210727/Centrality...
Processing 23 folder: ../../data/QuantLet/FRM_Quantlet_China/FRM_All/Output 20210727/Output...
Processing 24 folder: ../../data/QuantLet/FRM_Quantlet_China/FRM_All/Output 20210727/Output/Asia...
Processing 25 folder: ../../data/QuantLet/FRM_Quantlet_China/FRM_All/Output 20210727/Output/Asia/Empty...
Processing 26 folder: ../../data/QuantLet/FRM_Quantlet_China/FRM_All/Output 20210727/Output/Asia/Empty/Adj_Matrices_ShapleyCom...
Processing 27 folder: ../../data/QuantLet/FRM_Quantlet_China/FRM_All/Output 20210727/Output/Asia/Empty/Adj_Matrices_Shapley

In [29]:
print(code_file_counter)

4867


In [30]:
repos_df = pd.DataFrame(repos)
repos_df = repos_df[~((repos_df.code_script=='empty')&(repos_df.metainfo_file=='empty')&(repos_df.type_script=='empty'))]
#repos_df['q_name'] = repos_df.folder_name.str[26:]
repos_df = repos_df.drop_duplicates(['folder_name', 'script_name'])
print(repos_df.shape)

repos_df = repos_df[repos_df.code_script!='empty']
repos_df = repos_df.drop_duplicates(['folder_name', 'script_name'])
print(repos_df.shape)

(4867, 5)
(4864, 5)


In [31]:
# REMOVE DUPLICATES IPYNB PY (that were created intially)
repos_df['script_name_no_ext'] = repos_df.script_name.str.split('.', expand=True)[0]

py_scripts = repos_df[repos_df.type_script=='py'].script_name_no_ext
ipy_scripts = repos_df[repos_df.type_script=='ipynb'].script_name_no_ext

both = list(set(ipy_scripts).intersection(set(py_scripts)))
both = [f'{q}.py' for q in both]

repos_df = repos_df[~repos_df.script_name.isin(both)]
print(repos_df.shape)

(4837, 6)


In [32]:
NAME = f'../../data/preprocessed/Quantlet/Parsed_Qs_with_code_{date.today().strftime("%d%m%Y")}'
with open(f'{NAME}.pkl', 'wb') as file:
    pickle.dump(repos_df, file)

In [33]:

with open(f'{NAME}.pkl', 'rb') as file:
    repos_df = pickle.load(file)
print(repos_df.shape)

(4837, 6)


In [34]:
repos_df.type_script = repos_df.type_script.str.lower()
repos_df = repos_df[repos_df.type_script.isin(
            ['py', 'r', 'm', 'ipynb']
        )
    ]
repos_df = repos_df[~repos_df.folder_name.str.contains('.ipynb_checkpoints')]
print(repos_df.shape)

(4837, 6)


In [35]:
repos_df.loc[repos_df.type_script== 'ipynb', 'type_script']='py'

In [36]:
repos_df = repos_df.reset_index(drop=True)

In [37]:
NAME = f'../../data/preprocessed/Quantlet/Qs_reduced_{date.today().strftime("%d%m%Y")}'
with open(f'{NAME}.pkl', 'wb') as file:
    pickle.dump(repos_df, file)